In [22]:
MODEL = "qwen3:1.7b"

# User Query Input

In [23]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)
        
def multiple_answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None, n_answers=1):
    answers = []
    for _ in range(n_answers):
        answer = answer_this_prompt(prompt, stream=stream, model=model, temperature=temperature, format=format)
        answers.append(answer)
    return answers

In [24]:
# Example usage
ans = answer_this_prompt("What is the Big Bang theory?", stream=True)
print("------------------------")
# print(ans)

<think>
Okay, I need to explain the Big Bang theory. Let me start by recalling what I know. The Big Bang theory is a cosmological model that explains the early development of the universe. So, the key points are that the universe began from a singularity and has been expanding ever since.

Wait, but I should make sure I'm not mixing up anything. The theory suggests that the universe was once extremely hot and dense, and then it expanded. But what exactly happened in the first moments? There's the concept of the universe's expansion, and the expansion rate. Also, the cosmic microwave background radiation is a key piece of evidence supporting this theory.

I should mention the evidence like the redshift of galaxies, the distribution of galaxies, and the abundance of light elements. Oh, right, the abundance of light elements like hydrogen and helium is another piece. The theory also explains the observed expansion of the universe, which is supported by observations like the Hubble's Law.

But I need to structure this properly. Start with the basic idea, then the evidence, then the implications. Maybe also mention the timeline, like the age of the universe being about 13.8 billion years. Also, the fact that the universe is still expanding, and the role of dark energy in the current expansion.

Wait, but the user might be looking for a concise explanation. Let me check if I'm covering all the important aspects without going too deep. The Big Bang theory is the prevailing cosmological model, explaining the universe's origin. It's based on observations and theoretical models, including the expansion of the universe and the cosmic microwave background. The theory is supported by multiple lines of evidence, such as the observed expansion of galaxies, the abundance of light elements, and the cosmic microwave background.

I should also mention that the theory is not without controversy, but it's the most widely accepted explanation. Maybe include that it's a scientific theory, not a dogma, and that it's continually tested and refined with new data.

Hmm, I need to make sure the information is accurate. For example, the Big Bang theory doesn't exactly say the universe was created from a singularity, but that it began to expand from a very hot and dense state. The singularity is a concept in physics, but it's not a physical point, just a mathematical model.

Also, the expansion rate is important. The universe is expanding, and the rate of expansion is influenced by dark energy. The current expansion is accelerating due to dark energy, which is a key factor in the model.

I think that's the main points. Now, I need to put this together in a clear, concise way, using proper terminology and structure.
</think>

The **Big Bang Theory** is the prevailing scientific explanation for the universe's origin and evolution. It posits that the universe began as an extremely hot, dense singularity around 13.8 billion years ago and has since been expanding ever since. Here's a structured overview:

### Key Concepts:
1. **Origin**:  
   - The universe started as a singularity—a point of infinite density and temperature.  
   - It underwent a rapid expansion (inflation) followed by a slower, ongoing expansion.

2. **Expansion and Cooling**:  
   - The universe expanded, cooling as it spread out.  
   - This expansion explains the observed redshift of galaxies (Hubble's Law) and the distribution of galaxies in the universe.

3. **Evidence**:  
   - **Cosmic Microwave Background (CMB)**: The faint afterglow of the Big Bang, discovered in 1964, confirms the universe was once hot and dense.  
   - **Abundance of Light Elements**: The observed ratios of hydrogen, helium, and trace lithium match predictions from the Big Bang nucleosynthesis.  
   - **Galaxy Distribution**: The large-scale structure of the universe aligns with the theory's predictions of cosmic expansion.  
   - **Gravitational Lensing and Dark Energy**: Observations of the universe's expansion rate and acceleration (dark energy) support the model.

4. **Timeline**:  
   - The universe is approximately **13.8 billion years old**.  
   - The **inflationary epoch** (10^-36 seconds to 10^-32 seconds) explains the rapid expansion.  
   - The **Hubble Time** (13.8 billion years) is determined by the observed expansion rate.

5. **Implications**:  
   - The universe is **not static** but expanding, with the rate of expansion influenced by **dark energy** (a mysterious force accelerating the expansion).  
   - The theory is **supported by multiple lines of evidence**, though it remains **theoretical** and **continuously refined** as new data emerges.

### Summary:
The Big Bang Theory is a foundational framework in cosmology, describing the universe's origin, evolution, and current state. It is supported by empirical evidence and remains the most comprehensive explanation for the cosmos' early history. While it addresses the **origin** of the universe, it does not explain **everything** (e.g., the nature of dark matter or the ultimate fate of the universe), but it provides a robust and testable framework for understanding the cosmos.

------------------------


In [ ]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net

# berkeleyAdmissionNet = Net(bn_path+"Berkeley Admissions.neta")


In [31]:
CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")

BN = ""

for node in ChestClinicNet.nodes():
	# print(f"{node.name()} -> {[child.name() for child in node.children()]}")
	BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"
	
print(BN)

VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []



In [34]:
from pydantic import BaseModel

class Query(BaseModel):
  fromNode: str
  toNode: str

PROMPT = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""

fromNode = "Smoking"
toNode = "Cancer"
input = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode) 

ans = answer_this_prompt(input, format=Query.model_json_schema())
query = Query.model_validate_json(ans)
print(query)

fromNode='Smoking' toNode='Cancer'
